# I. Préparation

## 1. Comptage

Ecrire une fonction `count` qui prend en parametre une chaîne de charactere `u` (ou en fait n'importe quel itérable), et qui renvoie une liste de paires `(k,a)`  où `a` décrit l'ensemble des symboles de `u` et `k` est son nombre d'occurrences dans `u`.

Par exemple, si `u="aababababbabcccbcbccccddddaaaaaaaaaaaaaaaaaaaaaa"`, cela doit retourner 
`[(28, 'a'), (8, 'b'), (8, 'c'), (4, 'd')]`.

In [21]:
from collections import Counter
import re

def count(u):
    res = dict(Counter(list(u)))
    return [(res[k],k) for k in res]

count("aababababbabcccbcbccccddddaaaaaaaaaaaaaaaaaaaaaa")
    

[(28, 'a'), (8, 'b'), (8, 'c'), (4, 'd')]

## 2. File de priorité

Le but de cette partie est d'écrire une classe `Heap` qui gère une file de priorité sous forme d'un tas. La classe doit implanter les méthodes suivantes :
- `__init__(self, key = lambda x:x)` le constructeur, qui prend en argument une fonction `key` (par défaut l'identité) qui sera utilisée pour les comparaisons : au lieu de comparer deux données `x` et `y`, on comparera
toujours `self.key(x)` et `self.key(y)`.
- `add(self,x)` qui ajoute la donnée `x` dans la file de priorité et met à jour la représentation interne pour garder la structure de tas selon les clés `self.key()`.
- `extract(self)` qui renvoie la donnée de plus petite clé et l'enlève de la file de priorité (et met à jour la représentation interne pour garder la structure de tas selon les clés `self.key()`). S'il n'y a pas d'élément, cela renvoie une exception.

Vos méthodes `add(self,x)`  et `extract(self)`  doivent être de complexité $\mathcal{O}(\log n)$, où $n$ est le nombre de données actuellement dans la file de priorité.

Vous pouvez bien sûr ajouter toutes les méthodes que vous souhaitez.

In [72]:
class Heap:
    def __init__(self, key = lambda x:x):
        self.heap = [0] #Init the heap with one value
        self.key = key
        self.size = 0 #Current size of the heap
    
    def shift_up(self, i):
        # While the element is not the root or the left element
        Stop = False
        while (i // 2 > 0) and Stop == False:
            # If the element is less than its parent swap the elements
            if self.key(self.heap[i]) < self.key(self.heap[i // 2]):
                self.heap[i], self.heap[i // 2] = self.heap[i // 2], self.heap[i]
            else:
                Stop = True
            # Move the index to the parent to keep the properties
            i //= 2

    def add(self, x):
        if(self.size == 0):
            self.heap.append(x)
            self.size+=1
        else:
            self.heap.append(x)
            self.size+=1
            self.shift_up(self.size)

    def shift_down(self, i):
        # if the current node has at least one child
        while (i * 2) <= self.size:
            # Get the index of the min child of the current node
            mc = self.min_child(i)
            # Swap the values of the current element is greater than its min child
            if self.key(self.heap[i]) > self.key(self.heap[mc]):
                self.heap[i], self.heap[mc] = self.heap[mc], self.heap[i]
            i = mc
 
    def min_child(self, i):
        # If the current node has only one child, return the index of the unique child
        if (i * 2)+1 > self.size:
            return i * 2
        else:
            # Here in the current node has two children
            # Return the index of the min child according to their values
            if self.key(self.heap[i*2]) < self.key(self.heap[(i*2)+1]):
                return i * 2
            else:
                return (i * 2) + 1


    def extract(self):
        if(len(self.heap)==1):
             raise ValueError("Empty Heap")
        root = self.heap[1]
 
        # Move the last value of the heap to the root
        self.heap[1] = self.heap[self.size]
 
        # Pop the last value since a copy was set on the root
        *self.heap, _ = self.heap
 
        # Decrease the size of the heap
        self.size -= 1
 
        # Move down the root (value at index 1) to keep the heap property
        self.shift_down(1)
 
        # Return the min value of the heap
        return root

    def add_list(self, list):
        for i in list:
            self.add(i)
        
    def printHeap(self):
        for i in range(1, self.size+1):
            print(i)


Testez votre classe `Heap` en implémentant le tri par tas (en passant en paramètre la fonction `key` à utiliser, par défaut l'identité). Vous pouvez ensuite générer aléatoirement un tableau avec des paires `(x,y)` de nombres aléatoires et les trier selon leur première coordonnée, ou selon leur deuxième coordonnée (en initialisant `key` à `lambda x:x[1]`).

In [77]:
from random import randint

def tri_par_tas(key, lst):
    tas = Heap(key)
    tas.add_list(lst)
    res = list()
    while tas.size != 0:
        res.append(tas.extract())
    return res

 
L = []
while len(L) < 5:
    C = (randint(0, 100000), randint(0, 10000))
    if C not in L:
        L.append(C)
print(L)

print(tri_par_tas(lambda x:x[0], L))



[(51210, 5847), (48888, 7064), (90968, 8106), (74555, 6503), (59856, 6493)]
[(48888, 7064), (51210, 5847), (59856, 6493), (74555, 6503), (90968, 8106)]


## 3. Encodage de Huffman

Créer une fonction `huffmanTree(L)` qui prend en argument une liste `L` sous le format retourné par `count` de la première question, et qui retourne un arbre de Huffman correspondant à ces nombres d'occurrences. Vous avez le choix sur la façon dont l'arbre est encodé, mais vous devez utiliser `Heap` pour faire les sélections des deux arbres de poids minimal à chaque étape.

In [82]:
def huffmanTree(L):
    tas = Heap(lambda x:x[0])
    for occu, name in L:
        tas.add((occu,name,None,None))
    while tas.size != 1:
        noeud1 = tas.extract()
        noeud2 = tas.extract()
        tmp_noeud = (noeud1[0] + noeud2[0], None, noeud1, noeud2)
        tas.add(tmp_noeud)
    return tas.extract()

L = [(28, 'a'), (8, 'b'), (8, 'c'), (4, 'd')]

print(huffmanTree(L))



(48, None, (20, None, (8, 'b', None, None), (12, None, (4, 'd', None, None), (8, 'c', None, None))), (28, 'a', None, None))


Ecrire une fonction `huffmanToCode(T)` qui prend en argument un arbre de Huffman et qui retourne un dictionnaire `C` tel que `C[a]` est l'encodage binaire du symbole `a` dans l'arbre (sous forme d'une chaîne ou d'une liste de `0` et `1`).

In [ ]:
def huffmanToCode(T):
    pass

Ecrire une fonction `applyCode(C,u)` où `C` est un encodage comme retourné par `huffmanToCode` et `u` est un mot sur le même jeu de symboles et qui retourne la suite de `0` et de `1` associé à l'encodage alphabétique de `u`en utilisant `C` (sous forme de chaîne de caractères ou de liste). 

Testez soigneusement vos fonctions, en vérifiant, par exemple, que cela est compatible avec l'exemple du cours.

On souhaite maintenant appliquer la compression de `Huffman` au fichier `etranger.txt`. Calculez le taux de compression obtenu.

## 4. Encodage binaire

Ecrire une classe `ByteEncoder` qui possède deux listes internes : l'une, `self.buffer` permet de stocker jusqu'à 8 bits '0' ou '1', l'autre, `self.tab` est une liste d'entiers entre 0 et 255 correspondant à un encodage de 8 bits. Implantez les méthodes suivantes :
- `addBit(self,b)` ajoute un bit '0' ou '1' à `buffer`. Si le buffer atteint la taille 8, l'entier correspondant à l'encodage binaire du buffer est ajouté dans `tab` et le buffer est vidé.
- `addString(self,s)` ajoute une chaîne (ou une liste) de '0' et de '1' à buffer avec des appels répétés à `addBit`.
- `write(filename)` écrit le contenu de l'objet dans le fichier `filename`: les éléments de `tab` sont ajouté un par un, et si le buffer n'est pas vide il est complété par des '0' avant d'être ajouté. Attention à bien ouvrir le fichier en écriture binaire avec "wb" et pour écrire une liste L de nombres (entre 0 et 255) utiliser la méthode `write(bytes(L))`.

## 5. Finalisation de l'encodage de Huffman

Ecrire une fonction `encodeTree` qui encode un arbre de Huffman dans un ByteEncoder. On suppose que les feuilles de l'arbre sont des `bytes`, des séquences de 8 bits. Cela signifie que lorsqu'on lit le fichier à compresser, il faudra s'assurer de l'ouvrir en binaire avec 'rb'.

Ecrire une fonction `HuffmanEncoding(filename, outname)` qui prend en argument un nom de fichier `filename`, qui calcule l'arbre de Huffman et le message compressé associé, et qui l'écrit dans `outname`.

## 6. Décodage

Ecrire une fonction `HuffmanDecode(filename, outname)` qui ouvre le fichier de nom `filename` compressé selon la méthode de Huffman, qui le décompresse, et qui écrit le resultat dans le fichier `outname`. 